In [1]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.1 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from pytorch_tabnet.tab_model import TabNetRegressor
import torch


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_excel('/content/drive/MyDrive/my_data_DS317/train_update.xlsx')
df.drop(['gioitinh'], axis=1)
df.head()

,mssv,namsinh,gioitinh,drl,diem_tt,dtb_toankhoa,dtb_tichluy,sotc_tichluy,diemtbhk_1,diemtbhk_2,diemtbhk_3,diemtbhk_4,diemtbhk_5,diemtbhk_6,diemtbhk_7,diemtbhk_8
0,39B39397XPvAibaEXe/vGZ1xWLvpK7wMZlgBlCBs,19,0,75.198135,30.0,9.37,9.37,24,9.37,9.34,9.40,9.370000,9.370000,9.370000,9.370000,9.370000
1,3AB392AFXPvAibaEXe/NRwqjPAC0MKjf0QSiSLjM,17,0,85.000000,26.5,7.07,7.07,139,7.00,6.74,6.25,6.230000,7.150000,6.920000,6.970000,7.600000
2,BEC54CF0XPvAibaEXe/qZKhoIW7F1rbPON7lHJUG,19,0,82.000000,30.0,6.54,6.54,18,6.54,6.54,6.54,6.540000,6.540000,6.540000,6.540000,6.540000
3,43386D57XPvAibaEXe/nw3DhNQnHfzJZvI0o00SJ,20,0,75.655535,22.0,1.83,1.83,4,3.56,0.00,1.78,1.780000,1.780000,1.780000,1.780000,1.780000
4,8F1A725FXPvAibaEXe88DMpF1KL2K3HyZfgFjAif,19,0,100.000000,30.0,9.31,9.31,64,9.21,9.32,9.41,9.313333,9.313333,9.313333,9.313333,9.313333


In [6]:
df_test = pd.read_excel('/content/drive/MyDrive/my_data_DS317/test_update.xlsx')
df_test.drop(['gioitinh'], axis=1)
df_test.head()

,mssv,namsinh,gioitinh,drl,diem_tt,dtb_toankhoa,dtb_tichluy,sotc_tichluy,diemtbhk_1,diemtbhk_2,diemtbhk_3,diemtbhk_4,diemtbhk_5,diemtbhk_6,diemtbhk_7,diemtbhk_8
0,14EEA146XPvAibaEXe/2sgPA+OKAks3AV3zsZSFH,22,0,82.105913,26.762789,8.77,8.77,123,8.61,8.79,9.01,9.04,8.63,8.620,8.530,8.747143
1,4A2628C7XPvAibaEXe9urngX21O5XaWExeziRFls,22,1,82.105913,30.000000,8.42,8.42,130,9.11,8.64,7.78,8.60,8.45,8.300,9.300,8.597143
2,44634117XPvAibaEXe+WdrMfsV/1fSPAGTu3hQ5U,22,0,82.105913,22.350000,7.73,7.73,137,7.83,7.46,7.53,7.67,7.89,8.030,7.750,7.737143
3,8A7BA5ACXPvAibaEXe8FNnc+c+WuJHdW9KyIp7Nl,19,0,100.000000,30.000000,6.52,6.52,83,7.63,7.21,7.70,7.05,1.67,6.252,6.252,6.252000
4,A3A77666XPvAibaEXe/4WyTnp6RTwi1hkjCNtuCX,20,0,75.655535,24.250000,6.50,6.50,148,6.02,6.75,6.79,6.26,5.21,5.580,6.450,6.690000


In [7]:
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100 if np.all(y_true != 0) else np.nan
    mpe = np.mean((y_true - y_pred) / y_true) * 100 if np.all(y_true != 0) else np.nan
    r2 = r2_score(y_true, y_pred)
    return {"R2 Score": r2, "MSE": mse, "RMSE": rmse, "MAE": mae, "MAPE": mape, "MPE": mpe}

In [17]:
models = {
    "TabNetRegressor": TabNetRegressor(
        n_d=8, n_a=8, n_steps=3,
        #random_state=42
    )
}

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [18]:
initial_columns = ['namsinh', 'dtb_toankhoa', 'dtb_tichluy', 'sotc_tichluy', 'diemtbhk_1']

# Lặp qua từng cột làm y
results = []
current_X_columns = initial_columns.copy()

for i in range(len(initial_columns)+2, len(df.columns) - 1):
    if i +2 >= len(df.columns): break
    target_column = df.columns[i+2]  # Cột y hiện tại

    # Xác định X và y
    X_train = df[current_X_columns]
    y_train = df[target_column]

    X_test = df_test[current_X_columns]
    y_test = df_test[target_column]

    # Huấn luyện và đánh giá mô hình
    for model_name, model in models.items():
        model.fit(
            X_train.values, y_train.values.reshape(-1, 1),
            max_epochs=50,
            patience=10
        )

        # Dự đoán
        y_pred = model.predict(X_test.values)

        # Tính toán các độ đo
        metrics = calculate_metrics(y_test, y_pred)
        metrics["Model"] = model_name
        metrics["Target Column"] = target_column
        results.append(metrics)

    # Cập nhật X cho vòng tiếp theo
    current_X_columns.append(target_column)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 21.02646|  0:00:00s
epoch 1  | loss: 2.15993 |  0:00:01s
epoch 2  | loss: 1.17245 |  0:00:01s
epoch 3  | loss: 0.96477 |  0:00:02s
epoch 4  | loss: 0.89616 |  0:00:02s
epoch 5  | loss: 0.84157 |  0:00:03s
epoch 6  | loss: 0.85507 |  0:00:03s
epoch 7  | loss: 0.84078 |  0:00:04s
epoch 8  | loss: 0.83393 |  0:00:04s
epoch 9  | loss: 0.79875 |  0:00:05s
epoch 10 | loss: 0.79957 |  0:00:05s
epoch 11 | loss: 0.80391 |  0:00:06s
epoch 12 | loss: 0.80694 |  0:00:06s
epoch 13 | loss: 0.7946  |  0:00:07s
epoch 14 | loss: 0.7777  |  0:00:07s
epoch 15 | loss: 0.79054 |  0:00:08s
epoch 16 | loss: 0.77188 |  0:00:08s
epoch 17 | loss: 0.77438 |  0:00:09s
epoch 18 | loss: 0.78794 |  0:00:10s
epoch 19 | loss: 0.77582 |  0:00:11s
epoch 20 | loss: 0.76908 |  0:00:11s
epoch 21 | loss: 0.74357 |  0:00:12s
epoch 22 | loss: 0.77024 |  0:00:13s
epoch 23 | loss: 0.75928 |  0:00:14s
epoch 24 | loss: 0.75453 |  0:00:14s
epoch 25 | loss: 0.76237 |  0:00:15s
epoch 26 | loss: 0.75857 |  0:00:15s
e

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 23.58383|  0:00:00s
epoch 1  | loss: 3.02532 |  0:00:01s
epoch 2  | loss: 0.9904  |  0:00:01s
epoch 3  | loss: 0.89665 |  0:00:02s
epoch 4  | loss: 0.84635 |  0:00:02s
epoch 5  | loss: 0.79913 |  0:00:03s
epoch 6  | loss: 0.70249 |  0:00:03s
epoch 7  | loss: 0.70749 |  0:00:04s
epoch 8  | loss: 0.7123  |  0:00:04s
epoch 9  | loss: 0.67961 |  0:00:05s
epoch 10 | loss: 0.65666 |  0:00:05s
epoch 11 | loss: 0.61442 |  0:00:06s
epoch 12 | loss: 0.61185 |  0:00:06s
epoch 13 | loss: 0.62621 |  0:00:07s
epoch 14 | loss: 0.63133 |  0:00:07s
epoch 15 | loss: 0.6648  |  0:00:08s
epoch 16 | loss: 0.64723 |  0:00:08s
epoch 17 | loss: 0.62356 |  0:00:09s
epoch 18 | loss: 0.59994 |  0:00:10s
epoch 19 | loss: 0.59962 |  0:00:11s
epoch 20 | loss: 0.57628 |  0:00:11s
epoch 21 | loss: 0.60003 |  0:00:12s
epoch 22 | loss: 0.5967  |  0:00:13s
epoch 23 | loss: 0.59236 |  0:00:13s
epoch 24 | loss: 0.56754 |  0:00:14s
epoch 25 | loss: 0.5648  |  0:00:14s
epoch 26 | loss: 0.54816 |  0:00:15s
e

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


...

epoch 0  | loss: 30.26579|  0:00:00s
epoch 1  | loss: 3.57224 |  0:00:01s
epoch 2  | loss: 1.59628 |  0:00:01s
epoch 3  | loss: 1.24761 |  0:00:02s
epoch 4  | loss: 1.12719 |  0:00:02s
epoch 5  | loss: 1.16373 |  0:00:03s
epoch 6  | loss: 1.1501  |  0:00:03s
epoch 7  | loss: 1.08229 |  0:00:04s
epoch 8  | loss: 1.08035 |  0:00:04s
epoch 9  | loss: 1.07044 |  0:00:05s
epoch 10 | loss: 1.0505  |  0:00:05s
epoch 11 | loss: 1.02815 |  0:00:06s
epoch 12 | loss: 1.05172 |  0:00:06s
epoch 13 | loss: 1.02061 |  0:00:07s
epoch 14 | loss: 1.01762 |  0:00:08s
epoch 15 | loss: 1.02811 |  0:00:09s
epoch 16 | loss: 1.02176 |  0:00:09s
epoch 17 | loss: 1.00523 |  0:00:10s
epoch 18 | loss: 1.02854 |  0:00:11s
epoch 19 | loss: 1.03236 |  0:00:11s
epoch 20 | loss: 1.02232 |  0:00:12s
epoch 21 | loss: 1.04768 |  0:00:12s
epoch 22 | loss: 1.01807 |  0:00:13s
epoch 23 | loss: 1.01047 |  0:00:13s
epoch 24 | loss: 1.06374 |  0:00:14s
epoch 25 | loss: 1.08765 |  0:00:14s
epoch 26 | loss: 1.01583 |  0:00:15s
e

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 16.19335|  0:00:00s
epoch 1  | loss: 1.9667  |  0:00:01s
epoch 2  | loss: 1.03489 |  0:00:01s
epoch 3  | loss: 0.8829  |  0:00:02s
epoch 4  | loss: 0.85583 |  0:00:02s
epoch 5  | loss: 0.86042 |  0:00:03s
epoch 6  | loss: 0.74939 |  0:00:03s
epoch 7  | loss: 0.73986 |  0:00:04s
epoch 8  | loss: 0.71379 |  0:00:04s
epoch 9  | loss: 0.76558 |  0:00:05s
epoch 10 | loss: 0.79108 |  0:00:05s
epoch 11 | loss: 0.70685 |  0:00:06s
epoch 12 | loss: 0.68364 |  0:00:07s
epoch 13 | loss: 0.6943  |  0:00:08s
epoch 14 | loss: 0.67438 |  0:00:08s
epoch 15 | loss: 0.64323 |  0:00:09s
epoch 16 | loss: 0.64768 |  0:00:10s
epoch 17 | loss: 0.65861 |  0:00:10s
epoch 18 | loss: 0.67146 |  0:00:11s
epoch 19 | loss: 0.65159 |  0:00:11s
epoch 20 | loss: 0.64816 |  0:00:12s
epoch 21 | loss: 0.65054 |  0:00:13s
epoch 22 | loss: 0.64054 |  0:00:13s
epoch 23 | loss: 0.6069  |  0:00:14s
epoch 24 | loss: 0.61725 |  0:00:14s
epoch 25 | loss: 0.64302 |  0:00:15s
epoch 26 | loss: 0.62978 |  0:00:15s
e

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 23.61913|  0:00:00s
epoch 1  | loss: 3.57216 |  0:00:01s
epoch 2  | loss: 1.70981 |  0:00:01s
epoch 3  | loss: 1.61246 |  0:00:02s
epoch 4  | loss: 1.52045 |  0:00:02s
epoch 5  | loss: 1.45248 |  0:00:03s
epoch 6  | loss: 1.43483 |  0:00:03s
epoch 7  | loss: 1.38667 |  0:00:04s
epoch 8  | loss: 1.36667 |  0:00:04s
epoch 9  | loss: 1.39045 |  0:00:05s
epoch 10 | loss: 1.40186 |  0:00:06s
epoch 11 | loss: 1.44173 |  0:00:06s
epoch 12 | loss: 1.36723 |  0:00:07s
epoch 13 | loss: 1.319   |  0:00:08s
epoch 14 | loss: 1.31446 |  0:00:09s
epoch 15 | loss: 1.31156 |  0:00:09s
epoch 16 | loss: 1.31985 |  0:00:10s
epoch 17 | loss: 1.28226 |  0:00:11s
epoch 18 | loss: 1.26088 |  0:00:11s
epoch 19 | loss: 1.24564 |  0:00:12s
epoch 20 | loss: 1.25651 |  0:00:12s
epoch 21 | loss: 1.21731 |  0:00:13s
epoch 22 | loss: 1.2143  |  0:00:13s
epoch 23 | loss: 1.22134 |  0:00:14s
epoch 24 | loss: 1.19904 |  0:00:14s
epoch 25 | loss: 1.18024 |  0:00:15s
epoch 26 | loss: 1.19033 |  0:00:15s
e

In [19]:
# Tạo DataFrame kết quả
results_df = pd.DataFrame(results)
results_df.to_csv("tabnet_model_results.csv", index=False)

print(results_df)

   R2 Score       MSE      RMSE       MAE  MAPE  MPE            Model  \
0  0.820141  0.803587  0.896430  0.561779   NaN  NaN  TabNetRegressor   
1  0.889016  0.440653  0.663817  0.410897   NaN  NaN  TabNetRegressor   
2  0.788095  0.897884  0.947567  0.523216   NaN  NaN  TabNetRegressor   
3  0.859814  0.576141  0.759039  0.482460   NaN  NaN  TabNetRegressor   
4  0.778324  1.017483  1.008704  0.492588   NaN  NaN  TabNetRegressor   
5  0.895478  0.442563  0.665254  0.383286   NaN  NaN  TabNetRegressor   
6  0.711073  1.460685  1.208588  0.629215   NaN  NaN  TabNetRegressor   

  Target Column  
0    diemtbhk_2  
1    diemtbhk_3  
2    diemtbhk_4  
3    diemtbhk_5  
4    diemtbhk_6  
5    diemtbhk_7  
6    diemtbhk_8  


In [20]:
print(results_df['R2 Score'].mean())
print(results_df['MSE'].mean())
print(results_df['RMSE'].mean())
print(results_df['MAE'].mean())

0.8202772636253431
0.8055707996323241
0.878485668744158
0.4976345888535236
